<a href="https://colab.research.google.com/github/Theallkeeeymist/duplicate-question-pair-nlp/blob/main/duplicate_question_pairs_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
df=pd.read_csv("/content/train.csv")
df.shape

(404290, 6)

In [3]:
df=df.sample(175000, random_state=42)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1


In [5]:
df['question1']=[data if isinstance(data,str) else "" for data in df["question1"]]
df['question2']=[data if isinstance(data,str) else "" for data in df["question2"]]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175000 entries, 8067 to 72265
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            175000 non-null  int64 
 1   qid1          175000 non-null  int64 
 2   qid2          175000 non-null  int64 
 3   question1     175000 non-null  object
 4   question2     175000 non-null  object
 5   is_duplicate  175000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 9.3+ MB


In [7]:
contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

In [8]:
import regex as re

def preprocess(q):
  q=str(q).lower().strip()

  q=q.replace('%', ' percent ')
  q=q.replace('$', ' dollar ')
  q=q.replace('₹', ' rupee ')
  q=q.replace('€', ' euro ')
  q=q.replace('@', ' at ')

  q=q.replace('[math]', '')

  q = q.replace(',000,000,000 ', 'b ')
  q = q.replace(',000,000 ', 'm ')
  q = q.replace(',000 ', 'k ')
  q = re.sub(r'([0-9]+)000000000', r'\1b', q)
  q = re.sub(r'([0-9]+)000000', r'\1m', q)
  q = re.sub(r'([0-9]+)000', r'\1k', q)

  q_concat=[]

  for word in q.split():
      if word in contractions:
        word=contractions[word]

      q_concat.append(word)

  q=' '.join(q_concat)
  q=q.replace("'ve", " have")
  q=q.replace("n't", " not")
  q=q.replace("'re", " are")
  q=q.replace("'ll", " will")

  #remove html tags
  q=BeautifulSoup(q)
  q=q.get_text()

  pattern=re.compile('\W')
  q=re.sub(pattern, ' ', q).strip()

  return q

In [9]:
df['preprocess_question1']=df['question1'].apply(preprocess)
df['preprocess_question2']=df['question2'].apply(preprocess)

In [10]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,preprocess_question1,preprocess_question2
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,how do i play pokémon go in korea,how do i play pokémon go in china
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,what are some of the best side dishes for crab...,what are some good side dishes for buffalo chi...
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,which is more advisable and better material fo...,what is the best server setup for buddypress
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,how do i improve logical programming skills,how can i improve my logical skills for progra...
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,how close we are to see 3rd world war,how close is a world war iii


In [11]:
df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0
preprocess_question1,0
preprocess_question2,0


In [12]:
df.duplicated().sum()

np.int64(0)

In [13]:
# Distribution of dataset

print(df["is_duplicate"].value_counts())

is_duplicate
0    110161
1     64839
Name: count, dtype: int64


In [14]:
# Distribution in percentage

print((df["is_duplicate"].value_counts()/df["is_duplicate"].count())*100)

is_duplicate
0    62.949143
1    37.050857
Name: count, dtype: float64


In [15]:
# Repeated questions

qid=pd.Series(df['qid1'].tolist()+df['qid2'].tolist())
print(f"Number of unique questions: {np.unique(qid).shape[0]}")

Number of unique questions: 270135


In [16]:
# Feature Engineering

In [17]:
df['q1_len']=df['preprocess_question1'].str.len()

df['q2_len']=df['preprocess_question2'].str.len()

In [18]:
df['q1_num_words']=df['preprocess_question1'].apply(lambda row: len(row.split(" ")))
df['q2_num_words']=df['preprocess_question2'].apply(lambda row: len(row.split(" ")))

In [19]:
def common_words(row):
  w1=set(map(lambda word: word.lower().strip(), row['preprocess_question1'].split(" ")))
  w2=set(map(lambda word: word.lower().strip(), row['preprocess_question2'].split(" ")))
  return len(w1&w2)

In [20]:
df['word_common']=df.apply(common_words, axis=1)

In [21]:
def total_words(row):
   w1=set(map(lambda word: word.lower().strip(), row['preprocess_question1'].split(" ")))
   w2=set(map(lambda word: word.lower().strip(), row['preprocess_question2'].split(" ")))
   return len(w1)+len(w2)

In [22]:
df['word_total']=df.apply(total_words, axis=1)

In [23]:
df['word_share']=round(df['word_common']/df['word_total'],2)

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
#Advanced Features

from nltk.corpus import stopwords

def fetch_token_features(row):
  q1=str(row['preprocess_question1'])
  q2=str(row['preprocess_question2'])

  SAFE_DIV=0.0001

  STOP_WORDS=stopwords.words("english")

  token_features=[0.0]*8

  # Converting sentence into tokens
  q1_tokens=q1.split()
  q2_tokens=q2.split()

  if len(q1_tokens)==0 or len(q2_tokens)==0:
    return token_features

  # set of non stopwords
  q1_words=set([word for word in q1_tokens if word not in STOP_WORDS])
  q2_words=set([word for word in q2_tokens if word not in STOP_WORDS])

  # stopwords in questions
  q1_stops=set([word for word in q1_tokens if word in STOP_WORDS])
  q2_stops=set([word for word in q2_tokens if word in STOP_WORDS])

  commmon_word_count=len(q1_words.intersection(q2_words))
  common_stop_count=len(q1_stops.intersection(q2_stops))
  common_token_count=len(set(q1_tokens).intersection(set(q2_tokens)))

  token_features[0]=commmon_word_count/(min(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[1]=commmon_word_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[2]=common_stop_count/(min(len(q1_stops),len(q2_stops))+SAFE_DIV)
  token_features[3]=common_stop_count/(max(len(q1_stops),len(q2_stops))+SAFE_DIV)
  token_features[4]=common_token_count/(min(len(q1_tokens),len(q2_tokens))+SAFE_DIV)
  token_features[5]=common_token_count/(max(len(q1_tokens),len(q2_tokens))+SAFE_DIV)

  token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
  token_features[7]=int(q1_tokens[0]==q2_tokens[0])

  return token_features

In [26]:
token_features=df.apply(fetch_token_features, axis=1)

df["cwc_min"]=list(map(lambda x:x[0], token_features))
df["cwc_max"]=list(map(lambda x:x[1], token_features))
df["csc_min"]=list(map(lambda x:x[2], token_features))
df["csc_max"]=list(map(lambda x:x[3], token_features))
df["ctc_min"]=list(map(lambda x:x[4], token_features))
df["ctc_max"]=list(map(lambda x:x[5], token_features))
df["last_word_eq"]=list(map(lambda x:x[6], token_features))
df["first_word_eq"]=list(map(lambda x:x[7], token_features))

In [27]:
!pip install distance
import distance

In [28]:
# length based features
def fetch_length_features(row):
  q1=row['preprocess_question1']
  q2=row['preprocess_question2']

  length_features=[0.0]*3

  # Converting the sentence into Tokens:
  q1_tokens=q1.split()
  q2_tokens=q2.split()

  if len(q1_tokens)==0 or len(q2_tokens)==0:
    return length_features

  # Absolute lenght feature
  length_features[0]=abs(len(q1_tokens)-len(q2_tokens))
  # Average token length
  length_features[1]=(len(q1_tokens)+len(q2_tokens))/2

  strs=list(distance.lcsubstrings(q1,q2))
  if len(strs) > 0:
        length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
  else:
        length_features[2] = 0.0
  return length_features

In [29]:
length_features=df.apply(fetch_length_features, axis=1)

df['abs_len_diff']=list(map(lambda x:x[0], length_features))
df['mean_len']=list(map(lambda x:x[1], length_features))
df['longest_substr_ratio']=list(map(lambda x: x[2], length_features))

In [30]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,preprocess_question1,preprocess_question2,q1_len,q2_len,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,how do i play pokémon go in korea,how do i play pokémon go in china,33,33,...,0.749981,0.999975,0.999975,0.874989,0.874989,0.0,1.0,0.0,8.0,0.823529
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,what are some of the best side dishes for crab...,what are some good side dishes for buffalo chi...,52,50,...,0.399992,0.999975,0.666656,0.666659,0.545450,0.0,1.0,2.0,10.0,0.333333
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,which is more advisable and better material fo...,what is the best server setup for buddypress,95,44,...,0.000000,0.499988,0.249997,0.249997,0.124999,0.0,0.0,8.0,12.0,0.111111
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,how do i improve logical programming skills,how can i improve my logical skills for progra...,43,51,...,0.999975,0.666644,0.399992,0.857131,0.666659,0.0,1.0,2.0,8.0,0.272727
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,how close we are to see 3rd world war,how close is a world war iii,37,28,...,0.599988,0.333322,0.249994,0.571420,0.444440,0.0,1.0,2.0,8.0,0.344828


In [31]:
!pip install fuzzywuzzy

In [32]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
  q1=row['preprocess_question1']
  q2=row['preprocess_question2']

  fuzzy_features=[0.0]*4

  # fuzz ratio
  fuzzy_features[0]=fuzz.QRatio(q1,q2)
  # fuzz partial ratio
  fuzzy_features[1]=fuzz.partial_ratio(q1,q2)
  # token_sort_ratio
  fuzzy_features[2]=fuzz.token_sort_ratio(q1,q2)
  # token_set_ratio
  fuzzy_features[3]=fuzz.token_set_ratio(q1,q2)

  return fuzzy_features

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [33]:
fuzzy_features=df.apply(fetch_fuzzy_features, axis=1)

df['fuzz_ratio']=list(map(lambda x: x[0], fuzzy_features))
df['fuzz_partial_ratio']=list(map(lambda x: x[1], fuzzy_features))
df['token_sort_ratio']=list(map(lambda x: x[0], fuzzy_features))
df['token_set_ratio']=list(map(lambda x:x[3], fuzzy_features))

In [34]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,preprocess_question1,preprocess_question2,q1_len,q2_len,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,how do i play pokémon go in korea,how do i play pokémon go in china,33,33,...,0.874989,0.0,1.0,0.0,8.0,0.823529,88,88,88,90
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,what are some of the best side dishes for crab...,what are some good side dishes for buffalo chi...,52,50,...,0.545450,0.0,1.0,2.0,10.0,0.333333,73,72,73,73
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,which is more advisable and better material fo...,what is the best server setup for buddypress,95,44,...,0.124999,0.0,0.0,8.0,12.0,0.111111,35,43,35,24
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,how do i improve logical programming skills,how can i improve my logical skills for progra...,43,51,...,0.666659,0.0,1.0,2.0,8.0,0.272727,72,67,72,96
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,how close we are to see 3rd world war,how close is a world war iii,37,28,...,0.444440,0.0,1.0,2.0,8.0,0.344828,68,61,68,81


In [35]:
ques_df=df[['preprocess_question1','preprocess_question2']]
ques_df.head()

,preprocess_question1,preprocess_question2
8067,how do i play pokémon go in korea,how do i play pokémon go in china
368101,what are some of the best side dishes for crab...,what are some good side dishes for buffalo chi...
70497,which is more advisable and better material fo...,what is the best server setup for buddypress
226567,how do i improve logical programming skills,how can i improve my logical skills for progra...
73186,how close we are to see 3rd world war,how close is a world war iii


In [36]:
final_df=df.drop(columns=['id', 'qid1', 'qid2', 'question1', 'question2', 'preprocess_question1', 'preprocess_question2'])
print(final_df.shape)
final_df.head()

(175000, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
8067,0,33,33,8,8,7,16,0.44,0.749981,0.749981,...,0.874989,0.0,1.0,0.0,8.0,0.823529,88,88,88,90
368101,0,52,50,11,9,6,20,0.30,0.399992,0.399992,...,0.545450,0.0,1.0,2.0,10.0,0.333333,73,72,73,73
70497,0,95,44,17,8,2,25,0.08,0.000000,0.000000,...,0.124999,0.0,0.0,8.0,12.0,0.111111,35,43,35,24
226567,1,43,51,7,9,6,16,0.38,0.999975,0.999975,...,0.666659,0.0,1.0,2.0,8.0,0.272727,72,67,72,96
73186,1,37,28,9,7,4,16,0.25,0.749981,0.599988,...,0.444440,0.0,1.0,2.0,8.0,0.344828,68,61,68,81


In [37]:
final_df['is_duplicate']

,is_duplicate
8067,0
368101,0
70497,0
226567,1
73186,1
...,...
4814,0
329938,0
287993,0
229492,0


In [38]:
from sklearn.feature_extraction.text import CountVectorizer

#text merging
questions=list(ques_df['preprocess_question1'])+list(ques_df['preprocess_question2'])

cv=CountVectorizer(max_features=650, ngram_range=(1,3))
q1_arr,q2_arr=np.vsplit(cv.fit_transform(questions).toarray(),2)

In [39]:
temp_df1=pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index=ques_df.index)
temp_df=pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(175000, 1300)

In [40]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,640,641,642,643,644,645,646,647,648,649
8067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
368101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
226567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
final_df=pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(175000, 1323)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,640,641,642,643,644,645,646,647,648,649
8067,0,33,33,8,8,7,16,0.44,0.749981,0.749981,...,0,0,0,0,0,0,0,0,0,0
368101,0,52,50,11,9,6,20,0.30,0.399992,0.399992,...,0,0,0,0,0,0,0,0,0,0
70497,0,95,44,17,8,2,25,0.08,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
226567,1,43,51,7,9,6,16,0.38,0.999975,0.999975,...,0,0,0,0,0,0,0,0,0,0
73186,1,37,28,9,7,4,16,0.25,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0


In [42]:
final_df['is_duplicate']

,is_duplicate
8067,0
368101,0
70497,0
226567,1
73186,1
...,...
4814,0
329938,0
287993,0
229492,0


In [43]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(final_df.iloc[:,1:].values, final_df.iloc[:,0].values, test_size=0.2, random_state=42)

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

rf=RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test,y_pred)}')
print(f'precision: {precision_score(y_test, y_pred)}')
print(f'recall: {recall_score(y_test, y_pred)}')
print(f'f1 score: {f1_score(y_test, y_pred)}')

accuracy score: 0.8067428571428571
precision: 0.7515606479652311
recall: 0.724316502931993
f1 score: 0.7376871170402544


In [45]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

model=XGBClassifier(verbosity=3)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test,y_pred)}')
print(f'precision: {precision_score(y_test, y_pred)}')
print(f'recall: {recall_score(y_test, y_pred)}')
print(f'f1 score: {f1_score(y_test, y_pred)}')

[08:45:50] ======== Monitor (0): HostSketchContainer ========
[08:45:50] AllReduce: 0.135579s, 1 calls @ 135579us

[08:45:50] MakeCuts: 0.147402s, 1 calls @ 147402us

[08:45:57] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (140000, 1322, 185080000).
[08:45:57] DEBUG: /workspace/src/gbm/gbtree.cc:131: Using tree method: 0
[08:46:31] ======== Monitor (0): GBTree ========
[08:46:31] BoostNewTrees: 33.6707s, 100 calls @ 33670650us

[08:46:31] CommitModel: 0.00014s, 100 calls @ 140us

[08:46:31] ======== Monitor (0): HistUpdater ========
[08:46:31] BuildHistogram: 8.35539s, 500 calls @ 8355391us

[08:46:31] EvaluateSplits: 0.858701s, 600 calls @ 858701us

[08:46:31] InitData: 0.043114s, 100 calls @ 43114us

[08:46:31] InitRoot: 23.591s, 100 calls @ 23590996us

[08:46:31] LeafPartition: 3.9e-05s, 100 calls @ 39us

[08:46:31] UpdatePosition: 0.87082s, 600 calls @ 870820us

[08:46:31] UpdatePredictionCache: 0.047483s, 100 calls @ 47483us

[08

In [49]:
import joblib

In [50]:
joblib.dump(model, 'duplicate_question_xgboost_new.pkl')
joblib.dump(rf, 'duplicate_question_random_forest_new.pkl')

['duplicate_question_random_forest_new.pkl']